In [2]:
import torch

In [75]:
import torch.nn as nn

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


****
##### Testing mha 

In [91]:
from mha import muliheaded_attention

In [92]:
testing_mha=muliheaded_attention(4,2)

In [93]:
## Checking the module list output. We based on the number of heads we pass, we will have that many linear layers.
testing_mha.q_mapping

ModuleList(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

In [95]:
# A random tensor representing 1 image, 5 patches and embedding size of each patch to be 4
testing_tensor=torch.randn(2,5,4)
print(testing_tensor)
testing_mha(testing_tensor).shape

tensor([[[-0.2625,  0.2740, -0.2138,  0.6563],
         [ 0.2145, -0.5587,  0.0124,  0.6617],
         [ 1.4776,  0.0247,  0.1867,  0.4367],
         [-0.4658, -1.9022,  1.7393,  1.5262],
         [-0.6221, -0.2175,  0.5872, -0.6206]],

        [[ 0.1950,  0.6210, -1.5737,  1.7337],
         [-0.9654,  0.8480,  1.0328, -0.8368],
         [-1.5797, -1.4777,  0.1693,  0.0318],
         [ 0.4615,  1.2474,  0.1352,  0.2735],
         [-0.1720,  1.0442,  0.9636,  1.7562]]])
torch.Size([5, 4])
tensor([[-0.1146, -0.6354],
        [-0.0770, -0.6512],
        [ 0.1110, -0.7642],
        [-0.1764, -0.5866],
        [-0.1533, -0.6099]], grad_fn=<MmBackward0>)
tensor([[-1.2227, -0.0050],
        [-1.2072, -0.0021],
        [-1.1608,  0.0028],
        [-1.2190,  0.0178],
        [-1.0228,  0.0423]], grad_fn=<MmBackward0>)
torch.Size([5, 4])
tensor([[-0.4493, -0.4046],
        [-0.7134, -0.2643],
        [-0.5432, -0.3444],
        [-0.4751, -0.3930],
        [-0.5797, -0.3361]], grad_fn=<MmBackward

torch.Size([2, 5, 4])

### Output says that 2 heads will perform calculations on 5 patches where embedding size is 4. Since we have 2 heads, the embedding will be broken down into size of 2 and 2 for each head.

***

#### Testing out patchify function

In [105]:
from vit_model import patchify, ViT

# Simulating an input where we have 3 images, each has 3 color channels. Each image has shape 4,4
# testing_patchfy=torch.randn(3,3,4,4)
testing_patchfy=torch.randn(7,1,28,28)

# The output in this case implies, each image has been broken down into 4 patches. Each patch has embedding dim=12
patchify(testing_patchfy,7).shape

torch.Size([7, 49, 16])

***
#### Testing out Vit class

In [106]:
myvit=ViT()
# myvit(testing_patchfy).shape
re=myvit(testing_patchfy)

torch.Size([7, 49, 16])
torch.Size([7, 50, 8])


In [108]:
re[0].shape

torch.Size([50, 8])

In [100]:
test_cls=nn.Parameter(torch.ones(1,16))

cls_input=[torch.cat((test_cls,p_s),dim=0) for p_s in re]
    

In [101]:
torch.stack(cls_input)[0]

tensor([[ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
          1.0000e+00],
        [-1.1258e+00, -1.1524e+00, -2.5058e-01, -4.3388e-01,  1.3894e+00,
          1.5863e+00,  9.4630e-01, -8.4368e-01, -4.4622e-01,  7.4402e-01,
          1.5210e+00,  3.4105e+00,  8.4189e-01, -4.0003e-01,  1.0395e+00,
          3.5815e-01],
        [ 8.4871e-01,  6.9201e-01, -3.1601e-01, -2.1152e+00, -6.1358e-01,
          3.1593e-02, -4.9268e-01,  2.4841e-01, -1.5312e+00, -1.2341e+00,
          1.8197e+00, -5.5153e-01, -2.4600e-01,  2.3025e+00, -1.8817e+00,
         -4.9727e-02],
        [ 3.2227e-01, -1.2633e+00,  3.4998e-01,  3.0813e-01,  4.3970e-01,
          1.1241e-01,  6.4079e-01,  4.4116e-01, -5.6925e-01,  9.1997e-01,
          1.1108e+00,  1.2899e+00, -1.0450e+00, -9.5650e-01,  3.3532e-02,
          7.1009e-01],
        [ 1.1984e-01

In [102]:
len(cls_input)

7

In [103]:
myvit.patch_embedding_dim

16